In [66]:
import numpy as np
import pandas as pd
import urllib
import sagemaker
import boto3
from sklearn.model_selection import train_test_split
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import os
from sagemaker.serializers import CSVSerializer

In [9]:
bucket_name="machine-learning-hard-parikh"
my_region=boto3.session.Session().region_name

In [15]:
s3=boto3.resource('s3')
try:
    if my_region=='us-west-2':
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={
                             'LocationConstraint': my_region
                         })
    print("Bucket successfully created")
except Exception as e:
    print("Error:",e)

In [16]:
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://machine-learning-hard-parikh/xgboost-as-a-built-in-algo/output


In [18]:
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [47]:
train_data,test_data=train_test_split(model_data,test_size=0.3,random_state=42)
train_data.shape,test_data.shape

((28831, 61), (12357, 61))

In [52]:
pd.concat([train_data['y_yes'],train_data.drop(['y_yes','y_no'],axis=1)],axis=1).to_csv('train.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train=sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name,prefix),content_type='csv')

In [57]:
pd.concat([test_data['y_yes'],test_data.drop(['y_yes','y_no'],axis=1)],axis=1).to_csv('test.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test=sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name,prefix),content_type='csv')

In [58]:
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [59]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [61]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [62]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-04-06-00-03-01-188


2024-04-06 00:03:01 Starting - Starting the training job...
2024-04-06 00:03:25 Starting - Preparing the instances for training...
2024-04-06 00:03:59 Downloading - Downloading input data...
2024-04-06 00:04:14 Downloading - Downloading the training image...
2024-04-06 00:04:39 Training - Training image download completed. Training in progress..[2024-04-06 00:05:01.297 ip-10-0-117-247.us-west-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:

In [63]:
xgb_predictor=estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-04-06-00-21-40-209
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-04-06-00-21-40-209
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-04-06-00-21-40-209


-----!

In [79]:
test_data_array=test_data.drop(['y_no','y_yes'],axis=1).values
xgb_predictor.content_type='text/csv'
xgb_predictor.serializer=CSVSerializer()
predictions=xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array=np.fromstring(predictions[1:],sep=',')
predictions_array

array([0.08820307, 0.02847173, 0.08207948, ..., 0.03812492, 0.09187825,
       0.08279892])

In [80]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.9%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10810)    35% (158)
Purchase        9% (1091)     65% (298) 



In [81]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-04-06-00-21-40-209


[{'ResponseMetadata': {'RequestId': 'DAPZDVF1K0XBY08F',
   'HostId': 'y41Z3qW5KDtayos3BhWUR1/4dTfoIB5Q+c9lLbOJys8BJKNuVX27nWuPZKmOiq9bQGz/zhxbRjQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'y41Z3qW5KDtayos3BhWUR1/4dTfoIB5Q+c9lLbOJys8BJKNuVX27nWuPZKmOiq9bQGz/zhxbRjQ=',
    'x-amz-request-id': 'DAPZDVF1K0XBY08F',
    'date': 'Sat, 06 Apr 2024 00:35:55 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-04-06-00-03-01-188/profiler-output/system/incremental/2024040600/1712361900.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-04-06-00-03-01-188/debug-output/collections/000000000/worker_0_collections.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-04-06-00-03-01-188/debug-output/events/000000000020/000000000020_worker_0.tfev